# TO DO
1. we have webscrape the "metacrtics"  website for top 500 movies 

# Task 1 : Webscraping
 1.import required libaries 
    * requests : To retrived the Html of the source page
    * bs4      : To converted the obtained html which in the form of text into soup object in which we can retrive specific                      data like headers, paragraphs, etc.
    * lxml     : Helps bs4 to understand the html.
    * csv      : To write or read into csv file.

In [1]:
import requests
import lxml
import bs4
import csv 


2.Loading the featured page links into a link to hit them

In [2]:
urls =["https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc&page=1"]



3. Initialise the varaible that are used to store the data

In [12]:
movie_short_href={} # used to store the href values 

movie_details={}    # Used to store the final movie details as 
                    #   { 'movie_name':{'Director':['director_name'], 
                    #                   'Principal Cast' : [name_1, name_2 ...name_m], 
                    #                    'Cast':['name_1, name_2... name_n'] }}  note m>n.
            
failed_items=[]     # Used to store the movie if of those movie where we have a failed attempt.




4. Below process is to achieve the link for each movies from the main page

    result : now we have collection of end points of the movies.

In [13]:

index_num=0
for url in urls:
    result = requests.get(url,headers={'User-Agent': 'Mozilla/5.0'})
    soup = bs4.BeautifulSoup(result.text, "lxml")
    div_tag = soup.find_all('div', class_=["browse_list_wrapper one browse-list-large",
                                           "browse_list_wrapper two browse-list-large",
                                           "browse_list_wrapper three browse-list-large",
                                           "browse_list_wrapper four browse-list-large"])
    for item in div_tag:
        for element in item.find_all('td', class_='clamp-image-wrap'):
            index_num=index_num+1
            source=element.find_all('a' , href= True)[0]
            movie_short_href[index_num]=source['href'].replace('/movie/','')
    
    

5. This is method is to scrape individual movie details from its respective website, if there is any kind of failure while scraping we store the the movie id  in a list (failed_items).

In [22]:
def retrive_data_from_website(i):
    movie_link='https://www.metacritic.com/movie/'
    url = movie_link+movie_short_href[i]+'/details'
    result = requests.get(url,headers={'User-Agent': 'Mozilla/5.0'})
    if result.status_code != 200:
        failed_items.append(i)
    else:
        soup = bs4.BeautifulSoup(result.text,'lxml')
        tags = soup.find_all('table', class_='credits')
        wanted_columns=['Director', 'Principal Cast', 'Cast']
        movie_name = soup.find_all('div', class_='product_page_title oswald upper')[0].find_all('h1')[0].getText()
        meta_score = soup.find_all('a', class_ = 'metascore_anchor')[0] 
        final_details={}
        for table in tags:
            column = table.find_all('th', class_="person")[0].getText()
            if column in wanted_columns:
                details=[]
                for name in table.find_all('a', href=True):
                    details.append(name.getText().strip())
                final_details[column]=details
        if movie_name not in movie_short_href.values():
            movie_short_href[i] = movie_name
        movie_details[movie_short_href[i]] = final_details
    if i in failed_items:
        failed_items.remove(i)
    

6. Now we hit every movie link from our collection and check for any failed attempts, 
    if any there is any failed attempt, we now change their href value in the collection(movie_short_href).

In [23]:

for i in movie_short_href.keys():
    retrive_data_from_website(i)


1 The Lord of the Rings: The Return of the King
2 Carlos
3 Nomadland
4 Marriage Story
5 La La Land
6 Faces Places
7 Crouching Tiger, Hidden Dragon
8 The Servant
9 Call Me by Your Name
10 The Magnificent Ambersons
11 Reversal of Fortune
12 David Byrne’s American Utopia
13 Shoplifters
14 Shoah: Four Sisters
15 The 39 Steps
16 Duck Soup
17 Petite Maman
18 Days of Heaven
19 Toni Erdmann
20 Yi Yi
21 McCabe & Mrs. Miller
22 The Best Years of Our Lives
23 Stagecoach
24 Inside Llewyn Davis
25 Sherpa
26 There Will Be Blood
27 Badlands
28 The Last Picture Show
29 Do the Right Thing
30 Sita Sings the Blues
31 The Decline of Western Civilization
32 Rio Bravo
33 Brother's Keeper
34 Crumb
35 Spotlight
36 The Band Wagon
37 One More Time with Feeling
38 The Diving Bell and the Butterfly
39 Invasion of the Body Snatchers
40 The Class
41 Platoon
42 35 Shots of Rum
43 Apocalypse Now: Final Cut
44 Chinatown
45 The Florida Project
46 Timbuktu
47 Frenzy
48 Brief Encounter
49 Cool Hand Luke
50 Little Women
5

In [18]:
if len(failed_items)>0:
    for i in failed_items:
        url = 'https://www.metacritic.com/movie/'+movie_short_href[i]
        dummy = requests.get(url,headers={'User-Agent': 'Mozilla/5.0'})
        movie_short_href[i]=dummy.url[dummy.url.rindex('/')+1::]
        retrive_data_from_website(i)

7. Write into a csv file

In [64]:
len(movie_details.keys())

500

In [69]:
for i in  movie_short_href.values() :
    if i in movie_details.keys():
        pass
    else :
        print(i)
        

In [ ]:
with open("dict.csv", "w", newline="") as csv_file:  
    writer = csv.writer(csv_file)
    for key, value in movie_details.items():
        writer.writerow([key, value])



8. Read from a csv file

In [ ]:
with open('dict.csv') as csv_file:
    reader = csv.reader(csv_file)
    mydict = dict(reader)


# Load the variable during dev phase 

Need to delete this when the project is completed. copy the cell from variable_pro.ipynb and run it under so we can load movie_details and movie_short_href

# Task
1. Analyze how many times has each actor/actress appeared in these top 500 movies, analyze how many times has each director appeared in these top 500 movies, what can that tell you about their career?
 to do 
  First extract all cast into a list :
  a) just the Principal cast?
  b) Principal cast +  cast?
  
  Second make a register for cast with their count attached  
  a) list of tuples  [(cast1,count),(cast2, count)...........]
  b) dictionary {{cast1:count}, {cast2: count}......... }
  c) just list  [cast1, cast2....][count1, count 2...] or [cast1, count1, cast2, count2..........]
  

# List the names of the cast that have appeared in different movies
1.Filter all the cast from each movie and append them to the list.


In [59]:
cast_names_all={}
def add_castnames_count(cast_names):
    for name in cast_names:
        if name in cast_names_all.keys():
            cast_names_all[name]=cast_names_all[name]+1
        else:
            cast_names_all[name]=1
    
for movie_name,movies in movie_details.items():
    if 'Principal Cast' in movies.keys():
        add_castnames_count(movies['Principal Cast'])
    if 'Cast' in movies.keys():
        add_castnames_count(movies['Cast'])

        
    

In [63]:
{k: v for k, v in sorted(cast_names_all.items(), key=lambda item: item[1], reverse =True)}

{'Harvey Keitel': 8,
 'Robert De Niro': 8,
 'Marlon Brando': 7,
 'Daniel Day-Lewis': 7,
 'John Ratzenberger': 7,
 'Ralph Fiennes': 7,
 'Timothy Spall': 7,
 'Willem Dafoe': 7,
 'Jim Broadbent': 7,
 'Orson Welles': 6,
 'James Stewart': 6,
 'Lucas Hedges': 6,
 'Ward Bond': 6,
 'Lesley Manville': 6,
 'Saoirse Ronan': 6,
 'Joseph Cotten': 5,
 'Al Pacino': 5,
 'Robert Duvall': 5,
 'Humphrey Bogart': 5,
 'John Qualen': 5,
 'Ethan Hawke': 5,
 'Thomas Mitchell': 5,
 'Harrison Ford': 5,
 'Kyle Chandler': 5,
 'Paul Giamatti': 5,
 'Wallace Shawn': 5,
 'Caleb Landry Jones': 5,
 'Rooney Mara': 5,
 'Cate Blanchett': 5,
 'Andy Serkis': 5,
 'Adam Driver': 5,
 'Scarlett Johansson': 5,
 'Emma Stone': 5,
 'Michael Stuhlbarg': 5,
 'Michael Keaton': 5,
 'Max von Sydow': 5,
 'Fabrizio Rongione': 5,
 'Ray Collins': 4,
 'Rudy Bond': 4,
 'Sterling Hayden': 4,
 'Cary Grant': 4,
 'Mahershala Ali': 4,
 'Walter Catlett': 4,
 'George C. Scott': 4,
 'Karl Malden': 4,
 'Henry Fonda': 4,
 'Murray Alper': 4,
 'Brad Pitt

In [68]:
x=dict()
x[]